# Goal


> Hi guys, the course I mentioned before is cs231n, which covers both neural network and convolutional neural network  
> So we will start with the basics about neural network, loss function and optimizer for next Tuesday  
> And we will be using pytorch as the main framework  
> Ying 3/27

Source material: http://cs231n.stanford.edu/syllabus.html

# Nearest-Neighbor

Memorize all the labeled data, compute the L1 distance $d(I_1, I_2) = \sum_{p} \left| I_1^p - I_2^p \right|$, output the label with the smallest distance

In [1]:
# code adapted from lecture
import numpy as np


class NearestNeighbor:
    def train(self, X, y):
        """
        X is N x D examples
        Y is 1 x N labels
        """
        self.X_train = X
        self.y_train = y
    
    def predict(self, X):
        """
        X is N x D data
        """
        num_examples = X.shape[0]
        
        diffs = self.X_train[:,None] - X[None,:]
        distances = np.sum(np.abs(diffs), axis=2)
        labels = np.argmin(distances,axis=1)
        predictions = self.y_train[labels]
        
        return predictions

In [2]:
nn = NearestNeighbor()
train_data = np.array([[1,2], [2,1]])
train_label = np.array(['cat', 'dog'])
nn.train(train_data, train_label)
nn.predict(train_data)

array(['cat', 'dog'], dtype='<U3')

$O(1)$ train, $O(n)$ predict. This is bad. We can live with slow training, we don't want slow predicting.

Relatedly, we have

# k-nearest neighbors

Take the majority vote from $k$ nearest points (L2 distance)

In practice, never used for image recognition:

- slow
- distance on pixels uninformative
    - easy to cook up different images with same distance
- curse of dimensionality
- good teaching example though
    - you now have a hyperparameter, k
    - we can talk about train/valid/test sets
    - cross-validation: in n-fold, we split the data into n equal folds, use n-1 for training and 1 for validation, average performance over different folds
        - not actually used very frequently in deep learning

# Linear Classifier

[Source](http://cs231n.github.io/linear-classify/)

Lead-up to neural networks

The problem with all the nearest-neighbor approaches is that they need you to store and compare against **all** the data. This sucks.

What if instead, we used all the data to form a "template" to compare against? Then we can discard all the data and just keep the templates. Much faster.

Idea: template is a linear function $f: R^D \to R^k, f(x_i, W, b) = W x_i + b$, where $W$ represents weights and $b$ represents bias.

Some notes:

- Bias dimension: we can simplify this further by extending $x_i$ with a bias dimension of all 1's, then we just tack the bias onto the weight matrix.
- Normalization: we usually subtract the mean from everything in our input to rescale
- Limitation: because we're creating a template from our data, if you have horses facing left and horses facing right, the template becomes a 2 headed horse. Similarly, unable to deal with differences in color well.

So how do we pick the right $W$ and $b$? Actually, how do we define **right**?

# Prelude

In a nearest-neighbor approach, for every input we had to (1) memorize all the data and (2) compare against all the data. This is space and time inefficient. Furthermore, the model was fast to train but slow to use: we would prefer the opposite of slow to train and fast to use.

**Idea: what if we used all our data to form a "template"?**

Recall that we have linear classifiers of the form $f(x_i, W, b) = W x_i + b$, equivalently $f(x_i, W) = W x_i$, where $x_i$ is an input image and $W$ is our weights matrix.

Our weights matrix essentially IS our template. What does this mean?

- Normalization: we should normalize our input by subtracting the mean image from everything (reasons come later)
- Limitation: for a single linear classifier, if you're trying to recognize horses facing left and horses facing right, your template ends up being a 2-headed horse

But rolling with linear classifiers for now: how do we know when we have the right weights matrix? How do we define **right**?

# Loss functions

An objective way of measuring how wrong something is.

For every item in our training set, our linear function will predict some score, and we know what the actual item should be.

The loss function is a way to penalize the difference between the two, i.e. the further off your prediction, the higher your loss.

We will go over two common loss functions:

1. Multiclass Support Vector Machine
2. Softmax

## Multiclass Support Vector Machine Loss

We define loss of the $i$th prediction as $L_i = \sum_{j \neq y_i} \max(0, s_j - s_{y_i} + \delta)$

The score of the right class has to be greater than the score of the incorrect classes by at least $\delta$, otherwise we linearly add the loss.

Intuition: we don't want wrong predictions to come closer than $\delta$ to our right class

We might also see other types of penalty functions, e.g.

- **Hinge loss:** $\max(0, -)$
- **Squared Hinge Loss (L2-SVM)**: $\max(0, -)^2$


Recall: we wanted a loss function so that we can find out the "ideal" weight

Problem: the weight function is not unique! Suppose there exists a perfect $W$ with 0 loss. Then $2W$, $3W$, ... all have 0 loss too.

Solution: regularization

### Regularization

We regularize by preferring one set of weights over the other. The most common regularization approach is to add the L2 norm,

$R(W) = \sum_k \sum_l W_{k,l}^2$

The bigger your weights, the worse your score.

Hence our full multiclass SVM loss becomes

$L = \text{data loss} + \text{regularization loss}$  
$L = \frac{1}{N} \sum_i L_i + \lambda R(W)$

Question: why penalize larger weights instead of smaller ones?

Answer: less overfitting, reduce the influence of a single input dimension

In [3]:
# Example loss function

def L(X, y, W):
    """
    X: all training examples, N_features x N_examples
    y: array of all correct class indexes, N_examples long
    W: weights, N_classes x N_features
    
    DOESN'T INCLUDE REGULARIZATION
    
    returns: loss for each example
    """
    delta = 1.0 # hyperparameter - in practice can always be set to 1, since it counterbalances weights
    scores = W.dot(X) # N_classes x N_examples
    yi_scores = scores[np.arange(scores.shape[0]), y] # inside every range element, pick yth thing
    margins = np.maximum(0, scores - np.matrix(yi_scores).T + delta)
    margins[np.arange(X.shape[1]), y] = 0 # don't include in loss calculation
    loss = np.mean(np.sum(margins, axis=0))
    return loss

In [4]:
X = np.array([[1,2,3], [1,2,3]]) # 3 examples, 2 features
y = np.array([0,1,2]) # 3 known labels
W = np.array([[1,0],[0,1],[1,1]]) # 3 classes by 2 features
print(L(X,y,W))

2.3333333333333335


Other SVMs we can look at are

- binary: equivalent where classes=2
- OVA (one versus all): train a binary SVM per class
- AVA (all versus all): pairwise?
- structured: max margin(correct class, highest-scoring incorrect runner-up class)

## Softmax classifier

Generalized analogue of a logistic regression to multiple classes.

Compare:

- Multiclass SVM outputs raw class scores that don't really have a meaning
- Softmax will output normalized class probabilities

In softmax, we keep the same weight function, but replace hinge loss with cross-entropy loss:

$L = -\log \left( \frac{e^{f_{y_i}}}{\sum_j e^{f_j}} \right) = -log (s(v))$

where $s(v)$ is the softmax function that takes an arbitrary vector of real valued scores and squashes it into a vector of values between 0 and 1 that sums to 1.

We interpret the INPUT scores as unnormalized log probabilities for each class, and OUTPUT as a normalized class probability.

We can draw further comparisons to MLE or MAP, but that's probably beyond the scope of today.

Technicalities:

- numerical stability is a concern with the exponent. We can compute $\frac{C e^{f_{y_i}}}{C \sum_j e^{f_j}}$ instead, where $\log C = - \max_j f_j$.
- SVM uses hinge loss / max-margin loss
- softmax is technically cross-entropy loss, but people call it softmax because of the squashing up there

# Optimization

Okay, so now that we have a loss function, we know how well our linear models work. Now how do we start?

1. Random search: just try random weights. Kind of stupid, but it gives us a good starting point.
2. Random search in nearby direction: a little better
3. Gradient descent: find the direction that minimizes our loss, and head that way

Gradient descent sounds promising. We can either do it numerically or analytically with calculus.

In [5]:
def numerical_gradient(f, x):
    fx = f(x)
    grad = np.zeros(x.shape)
    step = 10e-6
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:
        ix = it.multi_index
        x_0 = x[ix]
        x[ix] += step
        fx_new = f(x)
        x[ix] = x_0
        grad[ix] = (fx_new - fx) / step
        it.iternext()
    
    return grad

In [6]:
X = np.array([[1,2,1], [1,2,2]]) # 3 examples, 2 features
y = np.array([0,1,2]) # 3 known labels
W_0 = np.array([[0.5,0],[0,0.5],[2,3]]) # 3 classes by 2 features

numerical_gradient(lambda W: L(X, y, W), W_0)

array([[ 0.33333333,  0.66666667],
       [-0.66666667, -0.33333333],
       [ 0.33333333,  0.        ]])

Problem: REALLY slow. Needs to iterate over every single feature per update.

Instead, we can compute it analytically by taking the derivative with respect to each $w_j$.

For example, our earlier Multiclass SVM had the following loss function:  
$L_i = \sum_{j \neq y_i} \max \left( 0, w_j^T x_i - w_{y_i}^T x_i + \Delta \right)$

Taking the derivative, we have:  
$\nabla_{w_{y_i}} L_i = - \left( \sum_{j \neq y_i} \mathbb{1} \left( w_j^T x_i - w_{y_i}^T x_i + \Delta > 0 \right) \right) x_i$  
$\nabla_{w_{j}} L_i = \mathbb{1} \left( w_j^T x_i - w_{y_i}^T x_i + \Delta > 0 \right) x_i$

i.e. we scale our input vector by the number of nonzero classes. That wasn't so bad.

Notes:

- a further optimization is Mini-Batch Gradient Descent, where you evaluate gradients over a smaller sample of your data (e.g. $256$ examples). 
    - if $n=1$, this is Stochastic Gradient Descent - usually not done because vectorized code is faster
- notice that after coming up with the gradient, we need to pick a step size to travel by
    - another hyperparameter, more of an art than a science

# Neural Networks

Instead of just having one linear function, let's have a whole bunch of functions and connect them together.

## Backpropagation

A way to compute gradients in small manageable parts via recursively applying the chain rule.

Consider a single variable function.

Instead of analytically coming up with the entire gradient - break everything into a computation graph.

The units for the nodes are your choice. But for example, plus, mul, max gates.

Then for each node, you know how the gradient works, so you just apply the chain rule.

There's a few neat ways to think of it:

- add is a distributor
- max is a router
- mul is a switcher

Essentially just math though.

Then for a multivar function, you'd just have lots of Jacobians flying around. But you really only care about the diagonal.

Implementation notes:

- each node should support a forward() / backward() API
    - forward(): compute, store and output the gradient
    - backward(): apply chain rule to get gradient of loss function

The most useful insights from neurons:

- activation function / firing
- stacking them in layers
- degree of connectedness

So for example, previously we had $f = Wx$

Now we can stack another layer of that, $f = W_2 \max(0, W_1 x)$

# Convolutional Neural Networks

There was a problem with our neural network formulation earlier.

If we view one pixel = one feature, then for a small 32x32x3 image, you need 3072 weights!

Simplifying assumption: input are images. Therefore, we only care about

- width: how wide the image is
- height: how tall the image is
- depth: how many channels the image has (e.g. RGB has 3, CMYK has 4)

(Note on notation: confusingly, sometimes depth of a neural network = number of layers)

We therefore can transform images in a coherent and managed way between layers, without having to fully connect everything.

## Layers

There are three main types of layers in building a CNN:

- Convolutional Layer
- Pooling Layer
- Fully-Connected Layer

### Convolutional Layer

Parameters: set of learnable filters

Each filter is spatially small across width/height but spans full depth

We slide the filter across the image, producing a 2D activation map.

The activation maps are stacked along the depth dimension, producing our output volume.


Hyperparameters:

- $F$, filter size: "receptive field of neuron", local connectivity
- depth: number of filters that are looking at the same region of the input
- $S$, stride: how smoothly we slide the filters over the image, wider stride => smaller output volume
- $P$, zero-padding: how many zeros we pad around as a border, we can use it to control size of output volume

Input volume size is usually $W$, formula for output size is $\frac{W−F+2P}{S}+1$.

e.g. a 7x7 input, a 3x3 filter, stride 1, pad 0 => $\frac{7-3+0}{1}+1 = 5$, we get a 5x5 output

If we wanted an output that was the same size as the input, we could have set pad 1.

**Parameter sharing**: simplifying assumption, if a feature is useful at (x1,y1), it should also be useful at (x2,y2). So you only have *depth* sets of parameters.
- sometimes not applicable, e.g. eye vs nose recognition, so we relax it sometimes and have Locally Connected Layers.

### Fully-Connected Layer

CONV layers and fully connected layers can be converted between each other.

### Pooling Layer

A pooling layer basically downsizes the spatial dimensions that we're working with. This is usually done with 3x3 filter with a stride of 2 or a 2x2 filter with a stride of 2.

Some people don't believe in pooling, instead preferring more CONV layers. To reduce dimensions, they suggest using larger strides once in a while. In particular,

> Discarding pooling layers has also been found to be important in training good generative models, such as variational autoencoders (VAEs) or generative adversarial networks (GANs). It seems likely that future architectures will feature very few to no pooling layers.


### Normalization Layer

In practice, normalization layers don't seem to be very useful / minimal contribution.

## Architecture


One of the most common architectures is 

`INPUT -> [[CONV -> RELU]*N -> POOL?]*M -> [FC -> RELU]*K -> FC`

However, recent research has seen some non-linear-layering which works out

In [7]:
# https://github.com/yunjey/pytorch-tutorial/blob/master/tutorials/02-intermediate/convolutional_neural_network/main.py

import torch 
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable


# Hyper Parameters
num_epochs = 5
batch_size = 100
learning_rate = 0.001

# MNIST Dataset
train_dataset = dsets.MNIST(root='./data/',
                            train=True, 
                            transform=transforms.ToTensor(),
                            download=True)

test_dataset = dsets.MNIST(root='./data/',
                           train=False, 
                           transform=transforms.ToTensor())

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)

# CNN Model (2 conv layer)
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.fc = nn.Linear(7*7*32, 10)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out
        
cnn = CNN()


# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cnn.parameters(), lr=learning_rate)

# Train the Model
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images)
        labels = Variable(labels)
        
        # Forward + Backward + Optimize
        optimizer.zero_grad()
        outputs = cnn(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [%d/%d], Iter [%d/%d] Loss: %.4f' 
                   %(epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.data[0]))

# Test the Model
cnn.eval()  # Change model to 'eval' mode (BN uses moving mean/var).
correct = 0
total = 0
for images, labels in test_loader:
    images = Variable(images)
    outputs = cnn(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()

print('Test Accuracy of the model on the 10000 test images: %d %%' % (100 * correct / total))

# Save the Trained Model
torch.save(cnn.state_dict(), 'cnn.pkl')

Processing...
Done!
Epoch [1/5], Iter [100/600] Loss: 0.1380
Epoch [1/5], Iter [200/600] Loss: 0.1122
Epoch [1/5], Iter [300/600] Loss: 0.0949
Epoch [1/5], Iter [400/600] Loss: 0.0673
Epoch [1/5], Iter [500/600] Loss: 0.1085
Epoch [1/5], Iter [600/600] Loss: 0.1074
Epoch [2/5], Iter [100/600] Loss: 0.0440
Epoch [2/5], Iter [200/600] Loss: 0.0188
Epoch [2/5], Iter [300/600] Loss: 0.1125
Epoch [2/5], Iter [400/600] Loss: 0.0363
Epoch [2/5], Iter [500/600] Loss: 0.0179
Epoch [2/5], Iter [600/600] Loss: 0.1303
Epoch [3/5], Iter [100/600] Loss: 0.0377
Epoch [3/5], Iter [200/600] Loss: 0.0771
Epoch [3/5], Iter [300/600] Loss: 0.0085
Epoch [3/5], Iter [400/600] Loss: 0.1492
Epoch [3/5], Iter [500/600] Loss: 0.0219
Epoch [3/5], Iter [600/600] Loss: 0.0538
Epoch [4/5], Iter [100/600] Loss: 0.0148
Epoch [4/5], Iter [200/600] Loss: 0.0082
Epoch [4/5], Iter [300/600] Loss: 0.0095
Epoch [4/5], Iter [400/600] Loss: 0.0205
Epoch [4/5], Iter [500/600] Loss: 0.0083
Epoch [4/5], Iter [600/600] Loss: 0.0